# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os, json
import glob
import psycopg2
import pandas as pd
from sql_queries import *
from tabulate import tabulate

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        for f in files:
            fname = os.path.join(root, f)
            if fname.endswith(".json"):
                all_files.append(os.path.abspath(fname))
    return all_files
    



# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:

filepath = "/home/nonso/Documents/data_engineering/project-template/data/song_data"

In [6]:
song_files = get_files(filepath)
print(song_files[0])

/home/nonso/Documents/data_engineering/project-template/data/song_data/A/A/C/TRAACFV128F935E50B.json


In [7]:
def get_first_data(files, *args):
    list_of_values = pd.read_json(files[0], lines=True)[[item for item in args]].stack().values.tolist()
    return list_of_values
    


In [8]:
def get_dataframe_of_first_data(files):
    df = pd.read_json(files[0], lines=True)
    return df
    

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
song_data = get_first_data(song_files, "song_id", "title", "artist_id", "year", "duration")
song_data

['SOBLGCN12AB0183212',
 'James (Hold The Ladder Steady)',
 'AR47JEX1187B995D81',
 1985,
 124.86485]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
artist_data = get_first_data(song_files, "artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude")
artist_data

['AR47JEX1187B995D81', 'SUE THOMPSON', 'Nevada, MO', 37.83721, -94.35868]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

In [13]:
filepath2 = "/home/nonso/Documents/data_engineering/project-template/data/log_data"

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [14]:
filepath = "/home/nonso/Documents/data_engineering/project-template/data/log_data"


In [15]:
log_files = get_files(filepath)

In [16]:
df = get_dataframe_of_first_data(log_files)
df.head(4)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Kenny G,Logged In,Chloe,F,53,Cuevas,256.57424,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Everlasting,200,1542412944796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
1,Randy Crawford,Logged In,Chloe,F,54,Cuevas,251.40200,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Rio De Janeiro Blue (Album Version),200,1542413200796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
2,Placebo,Logged In,Chloe,F,55,Cuevas,224.02567,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Breathe Underwater,200,1542413451796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
3,Poison The Well,Logged In,Chloe,F,56,Cuevas,184.60689,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Riverside,200,1542413675796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [17]:


df = df.loc[df['page'] == "NextSong"]
df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Kenny G,Logged In,Chloe,F,53,Cuevas,256.57424,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Everlasting,200,1542412944796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
1,Randy Crawford,Logged In,Chloe,F,54,Cuevas,251.40200,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Rio De Janeiro Blue (Album Version),200,1542413200796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
2,Placebo,Logged In,Chloe,F,55,Cuevas,224.02567,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Breathe Underwater,200,1542413451796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49


In [18]:

t = df.reset_index()
time_df = None
for index, row in t.iterrows():
    t = pd.to_datetime(df['ts'], unit='ms')
    time_data_temp = pd.concat([t, t.dt.hour, t.dt.day, t.dt.isocalendar().week, t.dt.month, t.dt.year, t.dt.weekday], axis=1)
    column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']
    time_data_temp.columns = column_labels
    time_df = time_data_temp
print(((time_df.head(2))))


               start_time  hour  day  week  month  year  weekday
0 2018-11-17 00:02:24.796     0   17    46     11  2018        5
1 2018-11-17 00:06:40.796     0   17    46     11  2018        5


In [19]:

time_df.head(5)

,start_time,hour,day,week,month,year,weekday
0,2018-11-17 00:02:24.796,0,17,46,11,2018,5
1,2018-11-17 00:06:40.796,0,17,46,11,2018,5
2,2018-11-17 00:10:51.796,0,17,46,11,2018,5
3,2018-11-17 00:14:35.796,0,17,46,11,2018,5
4,2018-11-17 00:17:39.796,0,17,46,11,2018,5


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [20]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()
    print(list(row))



[Timestamp('2018-11-17 00:02:24.796000'), 0, 17, 46, 11, 2018, 5]
[Timestamp('2018-11-17 00:06:40.796000'), 0, 17, 46, 11, 2018, 5]
[Timestamp('2018-11-17 00:10:51.796000'), 0, 17, 46, 11, 2018, 5]
[Timestamp('2018-11-17 00:14:35.796000'), 0, 17, 46, 11, 2018, 5]
[Timestamp('2018-11-17 00:17:39.796000'), 0, 17, 46, 11, 2018, 5]
[Timestamp('2018-11-17 00:20:55.796000'), 0, 17, 46, 11, 2018, 5]
[Timestamp('2018-11-17 00:27:01.796000'), 0, 17, 46, 11, 2018, 5]
[Timestamp('2018-11-17 00:28:07.796000'), 0, 17, 46, 11, 2018, 5]
[Timestamp('2018-11-17 00:31:21.796000'), 0, 17, 46, 11, 2018, 5]
[Timestamp('2018-11-17 00:37:14.796000'), 0, 17, 46, 11, 2018, 5]
[Timestamp('2018-11-17 00:43:50.796000'), 0, 17, 46, 11, 2018, 5]
[Timestamp('2018-11-17 00:54:52.796000'), 0, 17, 46, 11, 2018, 5]
[Timestamp('2018-11-17 00:58:34.796000'), 0, 17, 46, 11, 2018, 5]
[Timestamp('2018-11-17 01:05:31.796000'), 1, 17, 46, 11, 2018, 5]
[Timestamp('2018-11-17 01:08:49.796000'), 1, 17, 46, 11, 2018, 5]
[Timestamp

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [21]:
df = get_dataframe_of_first_data(log_files)
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]
user_df = user_df[user_df.userId != '']
user_df.head(3)

,userId,firstName,lastName,gender,level
0,49,Chloe,Cuevas,F,paid
1,49,Chloe,Cuevas,F,paid
2,49,Chloe,Cuevas,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [22]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [23]:
for index, row in df.iterrows():
    print(pd.to_datetime(row.ts, unit="ms"), row.userId)
    



2018-11-17 00:02:24.796000 49
2018-11-17 00:06:40.796000 49
2018-11-17 00:10:51.796000 49
2018-11-17 00:14:35.796000 49
2018-11-17 00:17:39.796000 49
2018-11-17 00:20:55.796000 49
2018-11-17 00:27:01.796000 49
2018-11-17 00:28:07.796000 88
2018-11-17 00:31:21.796000 49
2018-11-17 00:37:14.796000 49
2018-11-17 00:43:50.796000 49
2018-11-17 00:54:52.796000 49
2018-11-17 00:58:34.796000 49
2018-11-17 01:05:31.796000 49
2018-11-17 01:08:49.796000 49
2018-11-17 01:12:35.796000 49
2018-11-17 01:18:20.796000 49
2018-11-17 01:23:53.796000 49
2018-11-17 01:26:40.796000 49
2018-11-17 01:29:37.796000 49
2018-11-17 01:34:23.796000 49
2018-11-17 01:38:18.796000 49
2018-11-17 01:42:22.796000 49
2018-11-17 02:20:52.796000 10
2018-11-17 04:31:38.796000 54
2018-11-17 04:38:41.796000 54
2018-11-17 04:42:08.796000 54
2018-11-17 04:47:15.796000 54
2018-11-17 04:52:02.796000 54
2018-11-17 04:53:03.796000 54
2018-11-17 04:58:07.796000 54
2018-11-17 05:01:41.796000 54
2018-11-17 05:02:37.796000 33
2018-11-17

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [24]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (pd.to_datetime(row.ts, unit="ms"), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

ForeignKeyViolation: insert or update on table "songplays" violates foreign key constraint "songplays_start_time_fkey"
DETAIL:  Key (start_time)=(2018-11-17 10:16:35.796) is not present in table "time".


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.